Resizing the REAL and FAKE images to 256x256

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
from PIL import Image

# Set the path to your zip file in the local session storage
zip_path = '/content/Detect_AI_Art.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Set the paths to the "REAL" and "FAKE" folders
real_folder = '/content/REAL/'
fake_folder = '/content/FAKE/'

# Function to resize an image
def resize_image(image_path, output_path, size):
    with Image.open(image_path) as img:
        img = img.resize(size)
        img.save(output_path)

# Create folders to store the resized images
os.makedirs('/content/REAL_resized', exist_ok=True)
os.makedirs('/content/FAKE_resized', exist_ok=True)

# Resize images in the "REAL" folder
for filename in os.listdir(real_folder):
    if filename.endswith('.jpg'):
        image_path = os.path.join(real_folder, filename)
        output_path = os.path.join('/content/REAL_resized', filename)
        resize_image(image_path, output_path, (256, 256))

# Resize images in the "FAKE" folder
for filename in os.listdir(fake_folder):
    if filename.endswith('.jpg'):
        image_path = os.path.join(fake_folder, filename)
        output_path = os.path.join('/content/FAKE_resized', filename)
        resize_image(image_path, output_path, (256, 256))

print("Image resizing completed.")

Image resizing completed.


Simple Convolutional Neural Network Architecture to Predict Real vs. AI Images


In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set the paths to the resized image folders
resized_folder = '/content/'

# Set the image size and batch size
img_size = (256, 256)
batch_size = 32

# Set the subset size (number of samples to use for training)
subset_size = 1000

# Create data generators for training, validation, and testing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators for train and validation sets
train_generator = train_datagen.flow_from_directory(
    directory=resized_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['REAL_resized', 'FAKE_resized'],
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    directory=resized_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['REAL_resized', 'FAKE_resized'],
    subset='validation',
    shuffle=True,
    seed=42
)

# Create generator for the test set
test_generator = test_datagen.flow_from_directory(
    directory=resized_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['REAL_resized', 'FAKE_resized'],
    shuffle=False
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model on a subset of the data
train_steps = subset_size // batch_size
validation_steps = int(subset_size * 0.2) // batch_size

model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=10
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {test_acc:.4f}')

Found 2728 images belonging to 2 classes.
Found 682 images belonging to 2 classes.
Found 3410 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 145s 5s/step - loss: 1.0991 - accuracy: 0.5310 - val_loss: 0.5087 - val_accuracy: 0.8125
Epoch 2/10
31/31 [==============================] - 144s 5s/step - loss: 0.5013 - accuracy: 0.7732 - val_loss: 0.3316 - val_accuracy: 0.8750
Epoch 3/10
31/31 [==============================] - 146s 5s/step - loss: 0.3492 - accuracy: 0.8589 - val_loss: 0.4071 - val_accuracy: 0.7969
Epoch 4/10
31/31 [==============================] - 149s 5s/step - loss: 0.2513 - accuracy: 0.8891 - val_loss: 0.1790 - val_accuracy: 0.9219
Epoch 5/10
31/31 [==============================] - 147s 5s/step - loss: 0.2711 - accuracy: 0.8901 - val_loss: 0.1793 - val_accuracy: 0.9375
Epoch 6/10
31/31 [==============================] - 146s 5s/step - loss: 0.2736 - accuracy: 0.8740 - val_loss: 0.2542 - val_accuracy: 0.8854
Epoch 7/10
31/31 [===========